# Project: Wrangling and Analyze Data

## Data Gathering

In [ ]:
# Importing all necessary libraries
import pandas as pd
import numpy as np
import requests
import tweepy
import time
import json

1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [ ]:
# Reading file into dataframe
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
# Retrieving image prediction file
image_predictions_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

with open('image-predictions.tsv', 'wb') as image_predictions_file:
    gathered = requests.get(image_predictions_url)
    image_predictions_file.write(gathered.content)
  
# Reading file into dataframe
tweet_predictions = pd.read_csv('image-predictions.tsv', delim_whitespace=True)

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

*The below `tweepy.Client` instance references `credentials.py` to store the API connection token. I have not included this file in my submission for security reasons. In order for the code to function, a new `credentials.py` must be created with the following variable: <br>*

**bearer_token** = (bearer token here)

In [ ]:
# Importing connection token
import credentials

# Creating tweepy.Client instance 
client = tweepy.Client(bearer_token=credentials.bearer_token, wait_on_rate_limit=True)

In [ ]:
# Initializing fields for tweet_json write process
## start_time and count are to track the iteration of my for loops
start_time = time.time()
count = 0

# When using client, you have to grab the fields that you want to query
tweet_fields = ["public_metrics"]

# Initializing tweet_id list to store our tweet_ids for the successful queries
tweet_id = []

# Opening tweet_json with write privileges. 
## Iterating over each tweet_id from twitter_archive to find additional metrics via API call
### Each successful query is written to the txt file and the tweet_id is stored in tweet_id list
with open('tweet_json.txt', 'w') as file:
    for single_id in twitter_archive['tweet_id']:
        try:
            temp = client.get_tweet(id = single_id, tweet_fields = tweet_fields)
            tweet_data = temp.data
            for key in tweet_fields:
                if tweet_data.get(key):
                    file.write(json.dumps(tweet_data[key]))
                else:
                    pass
            file.write('\n')
            tweet_id.append(single_id)
            count = count + 1
            print(count)
        except Exception as e:
            print('No tweet found for {} with error message {}'.format(str(single_id), str(e)))

# Once loop finishes, a statement containing total run time is printed
end_time = time.time()
print('Process finished in {} seconds'.format(start_time-end_time))

In [ ]:
# Initializing empty lines list to capture the data from our tweet_json.txt file
lines = []

# Iterating over each line in the list, extracting 'retweet_count' and 'like_count', storing to lines
with open('tweet_json.txt', 'r') as txt:
    for i in txt:
        try:
            row = json.loads(i)                
            lines.append({"retweet_count":row['retweet_count'],
                         "favorite_count":row['like_count']})
        except Exception as e:
            print(str(e))

In [ ]:
# Creating columns for data frame
columns = ['tweet_id', 'retweet_count', 'favorite_count']

# Dataframe creation with lines and tweet_id data.  
tweet_metrics = pd.DataFrame(lines, columns = columns)
tweet_metrics['tweet_id'] = tweet_id

# Displays first 2 lines of frame for validity check
tweet_metrics.head(2)

In [ ]:
# Saving tweet_metrics to .csv
## If I reload my notebook, I will lose the frame, which would require me to make the API calls again
### I still have 'tweet_json', however I would lose the list of successfully queried tweet id's

tweet_metrics.to_csv("tweet_metrics.csv", index=False)

At the end of the gathering process we have the following data frames:

`twitter_archive`: Contains the main data for the **We Rate Dogs** tweets <br>
`tweet_predictions`: Contains prediction results from the neural network sampled off of **We Rate Dogs** <br>
`tweet_metrics`: Contains additional public metrics (retweet_count, favorite_count) for tweet data

In [ ]:
tweet_metrics = pd.read_csv('tweet_metrics.csv')

## Assessing Data

Below I will display some basic info about each data frame for analysis. I will attempt to find any obvious quality or tidiness issues. After I identify any potential issues, I will explore further using programmatic analysis to help define the issue scope.

 ### twitter_archive Data Assessment

In [ ]:
# Display twitter_archive dataframe for visual analysis
twitter_archive

In [ ]:
# Display the .info for twitter_archive
twitter_archive.info()

In [ ]:
twitter_archive.describe()

Notes:
1. href tags in `source`
2. null data (NaN) appearing as string 'None' in several columns
3. data type may need updating for timestamp fields
4. data type for `tweet_id` to object??
4. combine dog type columns?
5. non-names in `name`
6. only need original tweets. can remove replies and retweets
7. `expanded_urls` missing values
8. `rating_denominator` per schema should be /10. we have a min value of 0
9. `rating_numerator` has a **large** max value of 1776. is this legit?

#### `source` HTML Exploration

In [ ]:
# Counting unique values
twitter_archive.source.value_counts()

Notes: There are only 4 values in this column, which can be simplified to: 
1. Twitter for iPhone
2. Vine - Make a Scene
3. Twitter Web Client
4. TweetDeck

#### `name` Value Exploration

In [ ]:
# Attempting to determine 'bad names' by looking at unique values in name column
twitter_archive.name.value_counts().head(50)

In [ ]:
# Appears that proper names are capitalized
## Using regex to create a mask that will identify all strings in 'name' that start with a lower case letter
lower_mask = twitter_archive.name.str.contains('^[a-z]', regex = True)
twitter_archive[lower_mask].name.value_counts().sort_index()

In [ ]:
# Checking total count of lower case strings
len(twitter_archive[lower_mask])

In [ ]:
# Displaying with more concise columns for review
column_list = ['tweet_id', 'text', 'name']
twitter_archive[lower_mask][column_list]

Notes: Based on looking at the `text` fields, the incorrect `name` appears to come after 'is' in each example. The dog names do not appear in the tweet text, so we cannot just update the field with the correct name. Best solution is probably to set the invalid `name` values to `null`

Ex: `tweet_id` = 887517139158093824 <br>
`text` = 'I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy)'<br>
`name` = 'such'

#### `expanded_urls` Exploration

In [ ]:
# Display total number of entries missing a URL
sum(twitter_archive.expanded_urls.isna())

Since we can drop any rows that are replies or retweets, our actual count of null `expanded_urls` may be less than this

In [ ]:
# Creating mask to identify rows where `expanded_urls`, `in_reply_to_status_id`, and `retweeted_status_id` are all null
url_mask = (twitter_archive.expanded_urls.isna()) & (twitter_archive.in_reply_to_status_id.isna()) & (twitter_archive.retweeted_status_id.isna())

# Getting the length of the mask results
len(twitter_archive[url_mask])

In [ ]:
# Display records from url_mask
twitter_archive[url_mask]

Notes: I checked these tweets manually and confirmed they are not valid ratings, nor do any of them have images. I'm going to drop these for analysis

#### `rating_denominator` Exploration

In [ ]:
# Listing the counts of all denominator values to examine
twitter_archive.rating_denominator.value_counts().sort_index()

Notes:
1. We only have 1 tweet with a 0 value in the denominator. 
2. The majority of our tweets have '10' as the denominator. Most of the tweets without /10 appear to be multiples of 10. Should they actually be /10? 

In [ ]:
# Looking more closely at 0 denominator tweet
zero_mask = twitter_archive.rating_denominator == 0
twitter_archive[zero_mask]

Based on the `in_reply_to_status_id` being non-null, this tweet will be dropped from our data set when we clean anyway

In [ ]:
# Looking at tweets with non-10 values as the denominator 
## Excluding replies and retweets
denom_mask = (twitter_archive.rating_denominator != 10) & \
            (twitter_archive.in_reply_to_status_id.isna()) & \
            (twitter_archive.retweeted_status_id.isna())
                
twitter_archive[denom_mask]

In [ ]:
# Length of list with != 10 denominators
len(twitter_archive[denom_mask])

Note:

There are 17 tweets that contain a number other than 10 as the denominator (that are not replies or retweets)

In [ ]:
# Looking more closely at fields that may give us an explanation
column_list = ['tweet_id', 'text', 'rating_numerator', 'rating_denominator' ]
twitter_archive[denom_mask][column_list]

Notes:

Some of these denominators look like they are pulled from the wrong part of `text`

Ex:<br>
`tweet_id` = `722974582966214656` <br>
`text` = `Happy 4/20 from the squad! 13/10 for all`

The numerator and denominator are 4/20, however from the text we can clearly see that the rating should be 13/10. There are only a few of these, so we can manually update the numerator and denomenator columns with the correct values. 

Additionally, some of these tweets appear to have multiple dogs included (ie: puppers)

I'm not immediately sure how to handle instances of multiple dogs in an image. Since we are combining this frame with image breed predictions, it might make sense to just drop tweets with multiple dogs for our analysis. We have a small number of these compared to total tweets, it shouldn't impact our analysis too much. 

#### `rating_numerator` Exploration

In [ ]:
# Display numerator counts
twitter_archive.rating_numerator.value_counts().sort_index()

Notes:
1. Look at 0 values
2. Look at large values >= 15? 

In [ ]:
# Creating a mask to view numerators == 0
zero_mask = twitter_archive.rating_numerator == 0 

twitter_archive[zero_mask]

Note:

One of these tweets will be dropped because its a reply. The other tweet appears to be legitimate. Per the tweet text, that is the correct rating. I will leave as-is

In [ ]:
#Creating mask to view tweets with a `large` numerator >= 15 and denominator == 10
## Excluding retweets and replies
large_mask = (twitter_archive.rating_numerator >=15) & \
            (twitter_archive.rating_denominator == 10) & \
            (twitter_archive.in_reply_to_status_id.isna()) & \
            (twitter_archive.retweeted_status_id.isna())
twitter_archive[large_mask]

In [ ]:
# Length of 'large_mask'
len(twitter_archive[large_mask])

In [ ]:
twitter_archive[large_mask][column_list]

Notes: I chose 15 as my starting range since the majority of tweets have a numerator <= 15. Based on the tweet `text`, these values are either incorrectly converted decimals, or outliers. Since there are so few, I'm just going to drop them for analysis purposes

##### twitter_archive  notes:

Quality: 
1. source column contains unnecessary `<a href> </a>` tags 
2. `expanded_urls` contains 3 *bad* tweets
2. NaN data appearing as string `None` in several columns: name, doggo, floofer, pupper, puppo
3. `timestamp` and `retweeted_status_timestamp` should not be object, probably datetime
4. name column contains 109 non-names ex: `a`
5. 181 values exist in `retweeted*` fields. These should be removed
6. 78 replies exist in `in_reply_to_status_id` and `in_reply_to_user_id` fields. These should be removed
7. `tweet_id` should be 'object'? 
8. Invalid denominators should be manually corrected, or dropped
9. Invalid numerators should be dropped 


Tidiness: 
1. doggo, floofer, pupper, puppo can be combined to one column


### tweet_predictions Data Assessment

In [ ]:
# Display tweet_predictions dataframe for visual analysis
tweet_predictions

In [ ]:
# Displaying additional information on tweet_predictions
tweet_predictions.info()

Notes:

1. Our total count for **tweet_predictions** does not match the count for **twitter_archive**. This is likely due to tweets in the archive that have been deleted. We can't really do anything about this, and the missing data will **not** be handled in my cleaning phase 
2. `tweet_id` may be better off as an object, so we don't accidentally perform arithmatic on it
3. There are non-dog breeds in p* columns. 

#### `p*` Breed Exploration

In [ ]:
# Display boolean of True / False for 'dog_breed'
tweet_predictions.p1_dog.value_counts()

In [ ]:
# Display boolean of True / False for 'dog_breed'
tweet_predictions.p2_dog.value_counts()

In [ ]:
# Display boolean of True / False for 'dog_breed'
tweet_predictions.p3_dog.value_counts()

Notes: There are 500+ values across each p* column that are **not** actual dog breeds. We should check to see if there are any records where **none** of the 3 predictions are actual breeds

In [ ]:
# Create mask to find records where none of the 3 breeds are valid dog breeds
breed_mask = (tweet_predictions.p1_dog == False) & (tweet_predictions.p2_dog == False) & (tweet_predictions.p3_dog == False)
tweet_predictions[breed_mask]

Notes: 324 total records where none of the p* columns contain a valid dog breed. I spot checked a few of these records and confirmed that these appear to be correct, the image does not contain a dog. Since this isn't wrong, we can just include these with the rest of our variables when we combine tweet_predictions with twitter_archive

##### tweet_predictions  notes:

Quality: 
1. `p1`, `p2`, `p3` columns have mixed case strings
2. `p1`, `p2`, `p3` values have `_` seperated values. May be better to remove


Tidiness: 
1. p* columns can be combined to show the 'best guess' breed, and combined with twitter_archive frame

### tweet_metrics Data Assessment

In [ ]:
# Display tweet_metrics dataframe for visual analysis
tweet_metrics

In [ ]:
# Display additional information about metrics
tweet_metrics.info()

In [ ]:
# Find the difference in archive tweets and tweet metric values
print(len(twitter_archive.tweet_id) - len(tweet_metrics.tweet_id))

There are 29 fewer tweets in tweet_metrics than there are in twitter_archive. This is likely due to tweets from the archive that have been deleted. There isn't any need to drop these fields, they can just exist with null values for tweet_metrics. Best practice would be to combine this dataframe with twitter_archive 

Below is the final list of quality/tidiness issues that I will be cleaning. Any prior exploration or notation for issues **not** listed below will **not** be cleaned. There are many more opportunities to assess and clean beyond my final list, but per project specs this is not required beyond (8) Quality issues and (2) Tidiness issues

### Quality issues
1. **twitter_archive** - 181 retweets

2. **twitter_archive** - 78 reply tweets

3. **twitter_archive** - Non-names in `name` column

4. **twitter_archive** - Records with null value in`expanded_urls` 

5. **twitter_archive** - Unnecessary href tags in `source`

7. **twitter_archive** - `rating_denominator` + `rating_numerator` correct valid ratings manually

8. **twitter_archive** - `rating_denominator` records with multiple dogs

8. **twitter_archive** - `rating_numerator` records >= 15

### Tidiness issues
1. **twitter_archive** - Dog type (ie: doggo, floofer, pupper, puppo) should only be 1 column
2. **tweet_predictions** - p* columns can be combined to show the breed with highest confidence, and combined with **twitter_archive** frame
3. **tweet_metrics** - `retweet_count` and `favorite_count` can be combined to **twitter_archive**

## Cleaning Data

In [ ]:
# Make copies of original pieces of data
twitter_archive_clean = twitter_archive.copy()
tweet_predictions_clean = tweet_predictions.copy()
tweet_metrics_clean = tweet_metrics.copy()

### Quality Issue #1: 181 retweets

#### Define: Per project specs, we should only have original tweets in our final dataframe. The columns `retweeted_status_id`, `retweeted_status_user_id`, and `retweeted_status_timestamp` have a total 181 records that are non-null. This indicates retweets, or non-original tweets, in our dataframe. I am going to drop every record that has a non-null value in these columns 

#### Code

In [ ]:
# Verifying the correct number of retweets in the archive
print(len(twitter_archive_clean.retweeted_status_id.value_counts()))

In [ ]:
# Create a new copy of the clean dataframe with only null retweet values
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.retweeted_status_id.isna()]

In [ ]:
# Dropping freshly empty columns related to retweets
## Resetting the index to update after we dropped the retweets 
column_list = ['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp']
twitter_archive_clean.drop(columns =column_list, inplace = True)
twitter_archive_clean.reset_index(drop = True, inplace = True)

#### Test

In [ ]:
# Subtracting our new list len from the original len to confirm dropped row #
print(len(twitter_archive.tweet_id) - len(twitter_archive_clean.tweet_id))

In [ ]:
# Checking the new columns and index updates 
## I know that I don't have to reset the index until I'm done dropping rows
### This just helps everything feel more organized 
twitter_archive_clean.head(1)

### Quality Issue #2: 78 reply tweets

#### Define: Per project specs, we should only have original tweets in our final dataframe. The columns `in_reply_to_status_id` and `in_reply_to_user_id`,  have a total 78 records that are non-null. This indicates replies to tweets, or non-original tweets, in our dataframe. I am going to drop every record that has a non-null value in these columns 

#### Code

In [ ]:
# Verifying the correct number of replies in the archive
print(len(twitter_archive_clean.in_reply_to_status_id.value_counts()))

Note: Code above is stating 77 replies. This is 1 off from our original count. We may have dropped a tweet in our last clean step that was both a retweet *and* a reply. 

In [ ]:
# Create a new copy of our clean dataframe with only null reply values
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.in_reply_to_status_id.isna()]

In [ ]:
# Dropping freshly empty columns related to retweets
## Resetting the index to update after we dropped the retweets 
column_list = ['in_reply_to_status_id', 'in_reply_to_user_id']
twitter_archive_clean.drop(columns =column_list, inplace = True)
twitter_archive_clean.reset_index(drop = True, inplace = True)

#### Test

In [ ]:
# Subtracting our new list len from the original len to confirm dropped row #
print(len(twitter_archive.tweet_id) - len(twitter_archive_clean.tweet_id))

In [ ]:
# Checking the new columns and index updates 
## I know that I don't have to reset the index until I'm done dropping rows
### This just helps everything feel more organized 
twitter_archive_clean.head(1)

### Quality Issue #3: Non-names in `name` column

#### Define: In the data assessment phase, I confirmed that there are invalid names in the `name` column. Additionally, null values in `name` are represented as 'None'. Both of these issues will be cleaned in this step. Numpys .nan function will be used to change these 'bad' values to null. 

#### Code:

In [ ]:
# Creating a mask with regex to identify names that start with a lower case letter
name_mask = (twitter_archive_clean.name.str.contains('^[a-z]', regex = True)) | (twitter_archive_clean.name.str.contains('None'))

In [ ]:
# Using .loc to go through the values in the name column 
## Replacing any invalid names caught by name_mask with null, using numpy .nan function
twitter_archive_clean.loc[name_mask, 'name'] = np.nan

#### Test:

In [ ]:
# Checking to see any records with lowercase strings in 'name', or 'None'
twitter_archive_clean[name_mask].value_counts().sort_index()

### Quality Issue #4: Null values in `expanded_urls`

#### Define: In the assessment section, I identified 3 tweets that were missing `expanded_urls`, that were **not** replies or retweets. I checked each tweet manually and validated that these were not actual ratings, nor did they include any dogs. I feel the best solution for these 3 tweets is to remove them from our final data set. During clean, I will drop them. 

#### Code:

In [ ]:
# Creating a new copy of twitter_archive_clean that does not contain null url values
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.expanded_urls.notnull()]

#### Test:

In [ ]:
# Checking expanded_urls to see if any are still null
twitter_archive_clean.expanded_urls.isnull().value_counts()

### Quality Issue #5: href tags in `source`

#### Define: The `source` column contains unecessary href tags. Meaning, the text string contained within the href tags is legitimate, however there is no use for the tag itself. This can be simplified by extracting the actual value from these records and scrubbing the href tag. Using regex and .str.extract to grab the text string from inside the href tags.

There are only 4 values in this column:

1. Twitter for iPhone
2. Vine - Make a Scene
3. Twitter Web Client
4. TweetDeck

#### Code:

In [ ]:
# Getting a list of current source values with href tags
twitter_archive_clean.source.value_counts()

In [ ]:
# Ref: https://www.rexegg.com/regex-quickstart.html
## Using regex to extract the text value from inside the url tags
twitter_archive_clean.source = twitter_archive_clean.source.str.extract('^<a.+>(.+)</a>$')

#### Test:

In [ ]:
# Grabbing my source list again to confirm the str extract worked 
twitter_archive_clean.source.value_counts()

### Quality Issue #6: `rating_denominator` + `rating_numerator` correct valid ratings manually

#### Define: In the assessment phase, I discovered several tweets that did not have a demoninator of 10. Based on my visual analysis, I concluded that these ratings were not valid, and were caused by either: 
1. The script that originally grabbed the tweet archive data identified an incorrect rating in `text`. The correct rating exists in `text` as well. 
2. The rating is for a group of dogs

This step will be to clean issue #1. I will manually update the ratings for any applicable tweets below. If there is no valid rating in `text`, then the tweet will be dropped in the next clean step

#### Code:

In [ ]:
# Revisiting the list of != denominators in our clean data frame
column_list = ['tweet_id', 'text', 'rating_numerator', 'rating_denominator' ]
denom_mask = (twitter_archive_clean.rating_denominator != 10)
twitter_archive_clean[denom_mask][column_list]

In [ ]:
# Checking for any potential decminal values
twitter_archive_clean[twitter_archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")][['tweet_id','text', 'rating_numerator']]

Notes: 
1. tweet_id = 810984652412424192 : 
No valid rating
2. tweet_id = 740373189193256964 : 14/10
3. tweet_id = 716439118184652801 : 11/10
4. tweet_id = 682962037429899265 : 10/10
5. tweet_id = 666287406224695296 : 9/10
6. tweet_id = 883482846933004288 : 14/10
7. tweet_id = 786709082849828864 : 10/10
8. tweet_id = 778027034220126208 : 12/10

All other tweets in this list contain multiple dogs. There are 4 tweets able to be updated manually

In [ ]:
# Manually updating tweet_id with the corrected values in `rating_numerator` and `rating_denominator`
rating_mask = twitter_archive_clean.tweet_id == 740373189193256964
twitter_archive_clean.loc[rating_mask, 'rating_numerator'] = 14
twitter_archive_clean.loc[rating_mask, 'rating_denominator'] = 10

In [ ]:
# Manually updating tweet_id with the corrected values in `rating_numerator` and `rating_denominator`
rating_mask = twitter_archive_clean.tweet_id == 716439118184652801
twitter_archive_clean.loc[rating_mask, 'rating_numerator'] = 11
twitter_archive_clean.loc[rating_mask, 'rating_denominator'] = 10

In [ ]:
# Manually updating tweet_id with the corrected values in `rating_numerator` and `rating_denominator`
rating_mask = twitter_archive_clean.tweet_id == 682962037429899265
twitter_archive_clean.loc[rating_mask, 'rating_numerator'] = 10
twitter_archive_clean.loc[rating_mask, 'rating_denominator'] = 10

In [ ]:
# Manually updating tweet_id with the corrected values in `rating_numerator` and `rating_denominator`
rating_mask = twitter_archive_clean.tweet_id == 666287406224695296
twitter_archive_clean.loc[rating_mask, 'rating_numerator'] = 9
twitter_archive_clean.loc[rating_mask, 'rating_denominator'] = 10

In [ ]:
# Manually updating tweet_id with the corrected values in `rating_numerator` and `rating_denominator`
rating_mask = twitter_archive_clean.tweet_id == 883482846933004288
twitter_archive_clean.loc[rating_mask, 'rating_numerator'] = 14
twitter_archive_clean.loc[rating_mask, 'rating_denominator'] = 10

In [ ]:
# Manually updating tweet_id with the corrected values in `rating_numerator` and `rating_denominator`
rating_mask = twitter_archive_clean.tweet_id == 786709082849828864
twitter_archive_clean.loc[rating_mask, 'rating_numerator'] = 10
twitter_archive_clean.loc[rating_mask, 'rating_denominator'] = 10

In [ ]:
# Manually updating tweet_id with the corrected values in `rating_numerator` and `rating_denominator`
rating_mask = twitter_archive_clean.tweet_id == 778027034220126208
twitter_archive_clean.loc[rating_mask, 'rating_numerator'] = 12
twitter_archive_clean.loc[rating_mask, 'rating_denominator'] = 10

#### Test: 

In [ ]:
twitter_archive_clean[(twitter_archive_clean.tweet_id == 740373189193256964) | \
                     (twitter_archive_clean.tweet_id == 716439118184652801) | \
                     (twitter_archive_clean.tweet_id == 682962037429899265) | \
                     (twitter_archive_clean.tweet_id == 666287406224695296) | \
                     (twitter_archive_clean.tweet_id == 883482846933004288) | \
                     (twitter_archive_clean.tweet_id == 786709082849828864) | \
                     (twitter_archive_clean.tweet_id == 778027034220126208) ][column_list]

### Quality Issue #7: `rating_denominator` records with multiple dogs

#### Define: In the assessment phase, I discovered several tweets that did not have a demoninator of 10. Based on my visual analysis, I concluded that these ratings were not valid, and were caused by either: 
1. The script that originally grabbed the tweet archive data identified an incorrect rating in `text`. The correct rating exists in `text` as well. 
2. The rating is for a group of dogs

This step will be to clean issue #2. Any tweets that still exist with a `rating_denominator` != 10 are either multiples, or do not have a valid rating in the tweet `text`. These will be dropped as they are not helpful for analysis

#### Code:

In [ ]:
# Displaying the remaining tweets to drop
twitter_archive_clean[denom_mask][column_list]

In [ ]:
# Creating a new copy of the data frame with only records that have rating_denominator == 10
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.rating_denominator == 10]

#### Test:

In [ ]:
# Checking to see if any != 10 still exist
twitter_archive_clean[denom_mask][column_list]

In [ ]:
# Double checking by displaying the value counts for rating_denominator
twitter_archive_clean.rating_denominator.value_counts()

### Quality Issue #8: `rating_numerator` records >= 15

#### Define: In the assessment phase, I discovered several tweets that had a numerator rating >= 15. Upon further inspection, I confirmed that these tweets were either not valid (ie: not ratings), or they were nonsensical (even by WeRateDogs standards). I will drop these tweets in this step, as they are not useful for analysis. 

#### Code:

In [ ]:
# Displaying the tweets with rating_numerator >= 15
numerator_mask = (twitter_archive_clean.rating_numerator >= 15)
twitter_archive_clean[numerator_mask][column_list]

In [ ]:
# Creating a new copy of twitter_archive_clean with only numerators < 15
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.rating_numerator < 15]

#### Test:

In [ ]:
# Checking to see if any < 15 values still exist
twitter_archive_clean[numerator_mask][column_list]

In [ ]:
# Double checking by displaying the value counts
twitter_archive_clean.rating_numerator.value_counts().sort_index()

### Tidiness Issue #1: Combine dog type (ie: `doggo`, `floofer`, `pupper`, `puppo`) into `dog_type` 

#### Define: In the twitter_archive_clean dataframe, there are 4 columns representing a "type" of dog. This does not adhere to the rules of tidy data. This clean step will be to combine the following columns to one: 
1. `doggo`
2. `floofer` 
3. `pupper`
4. `puppo`

Additionally, instead of NaN appearing for null values, these columns contain text string 'None'. I will also be replacing these values with null

#### Code: 

In [ ]:
# Displaying DF for visual review
twitter_archive_clean.head()

In [ ]:
# Ref: https://stackoverflow.com/questions/39291499/how-to-concatenate-multiple-column-values-into-a-single-column-in-pandas-datafra
## Using .apply to join the dog type columns into singular column dog_type
column_list = ['doggo', 'floofer', 'pupper', 'puppo']
twitter_archive_clean['dog_type'] = twitter_archive_clean[column_list].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [ ]:
# Displaying updated DF for review
twitter_archive_clean.head(1)

In [ ]:
# Dropping the dog columns and resetting the index
twitter_archive_clean.drop(columns =column_list, inplace = True)
twitter_archive_clean.reset_index(drop = True, inplace = True)

In [ ]:
# Displaying a list of values in dog_type
twitter_archive_clean.dog_type.value_counts()

Notes: Looks like we have 11 tweets with more than one `dog_type`. Since we combined the columns uniformly, this is ok. I would like to go back through these tweets and remove any of the 'None' values. They are unnecessary (as they are null), and make the column look a little clunky

In [ ]:
# Replacing None_None_None_None with null, as these tweets contain no valid dog_type
twitter_archive_clean.loc[twitter_archive_clean.dog_type.str.contains('None_None_None_None'), 'dog_type'] = np.nan

In [ ]:
# Replacing the str to strip out erroneous 'none' values
twitter_archive_clean.dog_type = twitter_archive_clean.dog_type.str.replace('None_None_pupper_None', 'pupper')
twitter_archive_clean.dog_type = twitter_archive_clean.dog_type.str.replace('doggo_None_None_None', 'doggo')
twitter_archive_clean.dog_type = twitter_archive_clean.dog_type.str.replace('None_None_None_puppo', 'puppo')
twitter_archive_clean.dog_type = twitter_archive_clean.dog_type.str.replace('doggo_None_pupper_None', 'doggo_pupper')
twitter_archive_clean.dog_type = twitter_archive_clean.dog_type.str.replace('None_floofer_None_None', 'floofer')
twitter_archive_clean.dog_type = twitter_archive_clean.dog_type.str.replace('doggo_None_None_puppo', 'doggo_puppo')
twitter_archive_clean.dog_type = twitter_archive_clean.dog_type.str.replace('doggo_floofer_None_None', 'doggo_floofer')

*Notes: I tried to accomplish the above using .str.exract, but I couldn't figure out the regex syntax*

#### Test: 

In [ ]:
# Revewing the new dog_type values
twitter_archive_clean.dog_type.value_counts()

In [ ]:
# Checking table to ensure columns dropped ok
twitter_archive_clean.head(1)

### Tidiness Issue #2: Combine p* columns in tweet_predictions_clean into twitter_archive_clean as `breed`tweet_predictions

#### Define: - p* columns can be combined to show the breed with highest confidence, and added to twitter_archive_clean as `breed`. Ideally, I would like to take the prediction with the highest confidence and assign that to `breed`. Then, take the `breed` column and join it to twitter_archive_clean. <br>
Per the project specs, the `p1` column contains the breed with the highest confidence. Meaning, `p1_conf` is also the highest confidence level for each image. I am going to ignore values where dog != True

#### Code: 

In [ ]:
tweet_predictions_clean.head()

In [ ]:
# Ref: https://www.geeksforgeeks.org/numpy-select-function-python/
## Creating ordered lists for numpy select function 
conditions = [(tweet_predictions_clean.p1_dog == True),(tweet_predictions_clean.p2_dog == True),(tweet_predictions_clean.p3_dog == True)]
breed_list = [tweet_predictions_clean.p1, tweet_predictions_clean.p2, tweet_predictions_clean.p3]
conf_list = [tweet_predictions_clean.p1_conf,tweet_predictions_clean.p2_conf,tweet_predictions_clean.p3_conf]

In [ ]:
# Ref: https://www.geeksforgeeks.org/numpy-select-function-python/
## Using numpy select to grab values for new columns based on whether or not the breed is a dog
tweet_predictions_clean['breed'] = np.select(conditions, breed_list, default = np.nan)
tweet_predictions_clean['confidence'] = np.select(conditions, conf_list, default = np.nan)

In [ ]:
# Checking that numpy select worked, and we now have both columns populated
column_list = ['tweet_id', 'breed', 'confidence']
tweet_predictions_clean[column_list]

*Notes: Now that we have successfully created new columns to show the best possible breed prediction and confidence level, these columns need to be joined on the main dataframe, twitter_archive_clean, in order to maintain best tidiness standards of having a master dataframe*

In [ ]:
# Ref: https://datacarpentry.org/python-socialsci/11-joins/index.html#:~:text=We%20can%20join%20columns%20from,want%20using%20the%20how%20parameter.
## Using pandas merge to join the frames based on tweet_id
twitter_archive_clean = pd.merge(twitter_archive_clean, tweet_predictions_clean[column_list], how='left', on='tweet_id')

#### Test: 

In [ ]:
# Display twitter_archive_clean to confirm merge was successful
twitter_archive_clean.head()

In [ ]:
# Checking value counts
twitter_archive_clean.count()

### Tidiness Issue #3: Combine tweet_metrics_clean `retweet_count` and `favorite_count` to twitter_archive_clean

#### Define: `retweet_count` and `favorite_count` in tweet_metrics_clean can be joined with twitter_archive_clean. Best practices indicate that it is better to have as few dataframes as possible. Anything able to be merged, should be. Since there are only 2 (non-id) columns in this frame, it is better to join on our master frame, twitter_archive_clean

#### Code: 

In [ ]:
# Display tweet_metrics_clean for visual review
tweet_metrics_clean.head()

In [ ]:
# Ref: https://datacarpentry.org/python-socialsci/11-joins/index.html#:~:text=We%20can%20join%20columns%20from,want%20using%20the%20how%20parameter.
## Using pandas merge to join frames on tweet_id
twitter_archive_clean = pd.merge(twitter_archive_clean, tweet_metrics_clean, how='left', on='tweet_id')

#### Test: 

In [ ]:
# Confirm merge was successful 
twitter_archive_clean.head()

In [ ]:
# Checking value counts
twitter_archive_clean.count()

## Storing Data

In [ ]:
# Store master data frame to .csv
twitter_archive_clean.to_csv("twitter_archive_master.csv", index=False)

## Analyzing and Visualizing Data

In [ ]:
# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Reading our master file into a new frame 'df' for analysis
df = pd.read_csv("twitter_archive_master.csv")

### What is the most popular dog breed on WeRateDogs?

#### Define: The first insight I will be looking to gain, is a better understanding of which dog breeds tend to be the most popular on WeRateDogs. Since the breed list in our data set is dervived from a prediction list, we cannot assume that these insights will be 100% accurate. We also have many tweets that do not have a breed listed at all. However, the predictions were made via a neural network using a sample of 100 tweets from our dataset. The insights gleaned from analyzing breed information will likely still be meaningful <br>
There are 2 ideas to explore in this section. 
1. Which dog breed appears most frequently in the tweets
2. Which dog breed is the most favorited 

In [ ]:
# Show distinct breed count
print('There are {} distinct breeds'.format(len(df.breed.unique())))

In [ ]:
# Ref: https://stackabuse.com/seaborn-bar-plot-tutorial-and-examples/
## Create bar plot of the top 5 most popular breeds, by tweet frequency
pop_breeds = df.breed.value_counts().head(5)

plt.figure(figsize = (10,5))

pop_bar = sns.barplot(x=pop_breeds, y=pop_breeds.index, palette='hls')
pop_bar.set_title('WeRateDogs Top 5 Breed Listing')
pop_bar.set_xlabel('Tweet Frequency')
pop_bar.set_ylabel('Dog Breed');

In [ ]:
print('The top 5 dog breeds comprise {} % of total archived tweets'.format(round(sum(pop_breeds) * 100 / len(df.tweet_id))))

Based on the first visualization, it is apparent that Golden Retrievers are the most popular dog breed tweeted about by far. Followed by Labrador Retriever, Pembroke, Chihuahua, and finally Pug in fifth place. Again, this is based on a neural networks **prediction** of dog breed. This could mean that Golden Retriever owners are generally more active, or that a large number of WeRateDog followers own Golden Retrievers. There's no further insight to be gleaned from this, we can't determine the actual reason without additional studies. 

In [ ]:
# Creating variables fav_breeds and fav_breeds_sorted to group breed by favorites count
fav_breeds = df.groupby('breed')
fav_breeds = fav_breeds['favorite_count'].sum()
fav_breeds_sorted = fav_breeds.sort_values(ascending=False)
fav_breeds_sorted.head(5)

In [ ]:
print('The top 5 dog breeds comprise {}% of total favorite tweets'.format(round(sum(fav_breeds_sorted.head(5)) * 100 / sum(fav_breeds_sorted))))

If I group our favorite count by the breed association, I can see that again, Golden Retrievers are the most popular dog breed. At the time of this report there are 1,693,324 favorites on tweets with Golden Retrievers. The second most popular again is Labrador Retriever, third is Pembroke, and fourth is Chihuahua. This matches our previous analysis of most frequently occurring breeds. Fifth place is French Bulldog, which differs from our frequency count, where Pug lands in the fifth spot

### Where are users making the most popular tweets from?

#### Define: I would also like to know where users are tweeting from. I'd like to gain insight into not only the overall source popularity, but if there is any potential correlation in the popularity of the tweet (retweets + favorites) and which platform they were tweeted from

In [ ]:
# Check total number of tweets for each source
df.source.value_counts()

In [ ]:
# Group retweet and favorite count by platform
platform_group = df.groupby('source')[['retweet_count','favorite_count']].sum()
platform_group

In [ ]:
# Display percentage of tweets made from the top platform
print('Of every tweet in our archive, {}% were made from the top platform'.format(round(sum(df.source.value_counts().head(1)) * 100 / sum(df.source.value_counts()))))

In [ ]:
# Ref: https://seaborn.pydata.org/generated/seaborn.scatterplot.html
## Create a graph to explore the relationship between favorite / retweet count and platform
plt.figure(figsize = (10,5))

pop_platform = sns.scatterplot(x=df['retweet_count'], y=df['favorite_count'], hue=df['source'], palette='hls')
pop_platform.set_xlabel('Number of Retweets')
pop_platform.set_ylabel('Number of Favorites')
pop_platform.set_title('Tweet Popularity by Platform');

In [ ]:
# Displaying the scatterplot above as a table
column_list = ['tweet_id','source','retweet_count','favorite_count']
df[column_list].sort_values(by=['retweet_count', 'favorite_count'], ascending=False)

When looking at the analysis above, it becomes clear that the most popular platform to tweet from is **Twitter for iPhone**, then **Vine - Make a Scene**, **Twitter Web Client**, and finally **TweetDeck**. This also happens to be the order for our most popular tweets. The most popular tweets (by favorite_count and retweet_count), all come from **Twitter for iPhone**. 

It appears that overall, the most popular tweets are generated on the most popular platform. Correlation does not imply causation, there is no immediate reason to assume that tweets become popular *because* they were created on an iPhone. However, it is interesting to note. 

### What are the most popular dog names by frequency?

#### Define: I am also looking to gain insight into the popularity of certain dog names. Are there dog names that appear much more often than others? 

In [ ]:
# Display percentage of popular names / total name count
name_counts = df['name'].value_counts().sum()
top_five = df['name'].value_counts().head(5).sum()

print('The most popular dog names account for {}% of total names'.format(round(top_five * 100/ name_counts)))

In [ ]:
# Display number of occurances for most popular dog name
print('The most popular dog name has {} occurrances'.format(df.name.value_counts().head(1).sum()))

In [ ]:
# Display the list of 5 most popular dog names
print('The top 5 most popular dog names are:')
df['name'].value_counts().head(5)

Based on the analysis above, it does appear that there are a few names that occur more frequently than others. We have a large number of tweets that do not include a name, so my analysis was only based off of tweets that did include names. The most popular dog names by frequency are: **Lucy**, **Charlie**, **Oliver**, **Cooper**, and **Penny**. Even though these names appeared the most frequently, I am not sure if it is statistically significant. The top 5 names only comprised 4% of the total tweet list. 

### Insights:
1. **Golden Retrievers** are the overall most popular dog, both in frequency of tweets, and in favorited tweets. 
2. The most popular tweets are created on **Twitter for iPhone**, which also happens to be where the most tweets are sent from
3. The most popular names do not appear to be statistically significant when compared against the total number of tweets